In [1]:
import json
from tqdm import tqdm
import numpy as np
import pandas as pd
from joblib import load
import matplotlib
matplotlib.use('TKAgg')
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 18})
import plotly.graph_objects as go
import os
from inverse import *

import sys
np.set_printoptions(threshold=sys.maxsize, edgeitems=30, linewidth=100000, precision=2)

EPSILON = 1e-6

In [2]:
def create_dir(path):
    if not os.path.isdir(path):
        os.mkdir(path)

In [3]:
# Load configuration
current_dir = os.getcwd()

config_file_path = os.path.join(current_dir, 'config_regression.json')

with open(config_file_path, 'r', encoding='utf-8') as file:
    config = json.load(file)

reg = load('results_rf_regression/forest_40.joblib')
des_var_bounds = np.load('results_rf_regression/des_var_bounds.npy')

raw_data = pd.read_csv(config['training_data_path'], sep=',')
des_var_names = config['design_variable_names']
n_des_vars = len(des_var_names)

estimators = reg.estimators_
n_estimators = len(estimators)

# Inverse Inference

In [4]:
## Define target

# target = {
#     "type": "prescribed curve",
#     "prescribed curve": np.array(
#         [
#             0.03, 0.06, 0.09, 0.12, 0.15, 
#             0.18, 0.21, 0.24, 0.27, 0.3, 
#             0.3, 0.3, 0.3, 0.3, 0.3, 
#             0.3, 0.3, 0.3, 0.3, 0.3,
#             0.33, 0.35, 0.38, 0.40, 0.43,
#             0.46, 0.48, 0.52, 0.54, 0.58
#         ]
#     )
# }



In [10]:
target1 = {
    "type": "snap-through",
    "threshold": 0.2,
    "tol_threshold": 0.25,
    "stroke": 1.3,
    "tol_stroke": 0.25
}

grids1, prob1 = calculate_proba_in_des_space(des_var_bounds, target1, reg, 15, 0)

import pickle

with open("inverse_target_tol_25.pkl", "wb") as f:
    pickle.dump({"target": target1, "grids": grids1, "prob": prob1}, f)



In [11]:
target2 = {
    "type": "snap-through",
    "threshold": 0.2,
    "tol_threshold": 0.15,
    "stroke": 1.3,
    "tol_stroke": 0.15
}

grids2, prob2 = calculate_proba_in_des_space(des_var_bounds, target2, reg, 15, 0)

with open("inverse_target_tol_15.pkl", "wb") as f:
    pickle.dump({"target": target2, "grids": grids2, "prob": prob2}, f)
